<img src="2025-01-27-20-25-50.png" width=50%>

In [3]:
from langchain_ollama import ChatOllama

llm_structured_output = ChatOllama(model="llama3.2", format="json", temperature=0)

llm_structured_output.invoke("Hi")

AIMessage(content='{}', response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T20:28:31.077807Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 774494875, 'load_duration': 571586084, 'prompt_eval_count': 26, 'prompt_eval_duration': 169000000, 'eval_count': 2, 'eval_duration': 32000000}, id='run-85681c45-fbfd-4195-9260-a99df6941f38-0', usage_metadata={'input_tokens': 26, 'output_tokens': 2, 'total_tokens': 28})

In [23]:
import os

def create_dir(folder_path):
    """
    Creates a directory given a folder path.
    """
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    
    return f"Folder path was created at: {folder_path}"


create_dir("lucas-test-agents")


'Folder path was created at: lucas-test-agents'

In [5]:
!ls -d */

WebRover/          lucas-test-agents/ react-voice-agent/


In [6]:
def create_file(file_path, contents=""):
    """
    Creates a file with content.
    If no content is provided it will create an empty file.
    """
    
    with open(file_path, "w") as f:
        f.write(contents)
    
    return f"A file was created at: {file_path}"

create_file("./lucas-test-agents/file-text.txt", "This is a test")

'A file was created at: ./lucas-test-agents/file-text.txt'

In [7]:
def read_file(file_path):
    """
    Reads from file given its path.
    """
    
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file("./lucas-test-agents/file-text.txt")

'This is a test'

In [24]:
tools = [create_dir, create_file, read_file]

llm_tools = llm_structured_output.bind_tools(tools)

In [52]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = f"""You are an agent that helps users with desktop tasks like reading and writing files and creating directories. You either call a tool
from the options available: create_dir, create_file or read_file, or you return a summary of the tasks performed and at the end a string: END.
"""
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])
llm_agent = prompt_template | llm_tools

In [55]:
llm_agent.invoke("Create file named lucas-locas-pancakes.md")

AIMessage(content='', response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T21:30:19.116521Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_file', 'arguments': {'contents': '', 'file_path': 'lucas-locas-pancakes.md'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 438141625, 'load_duration': 26659959, 'prompt_eval_count': 330, 'prompt_eval_duration': 130000000, 'eval_count': 30, 'eval_duration': 279000000}, id='run-2a61ea11-f765-4ea6-842e-182bc2edb27a-0', tool_calls=[{'name': 'create_file', 'args': {'contents': '', 'file_path': 'lucas-locas-pancakes.md'}, 'id': '8d96d871-6bc1-465b-9165-38233700957e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 330, 'output_tokens': 30, 'total_tokens': 360})

In [56]:
output_tool_call = llm_agent.invoke("Create a file named ./test-agent.txt")
output_tool_call.tool_calls

[{'name': 'create_file',
  'args': {'contents': '', 'file_path': './test-agent.txt'},
  'id': 'ae06a878-1490-4aaa-a766-9488c5521aed',
  'type': 'tool_call'}]

In [57]:
tool_mapping = {
    "create_file": create_file,
    "create_dir": create_dir,
    "read_file": read_file
}

In [68]:
def llm_call(query, observations=[], actions_taken=[]):
    """
    Calls the llm, it can return a tool call with arguments for calling different tools,
    or an output to the user.
    """
    template = """
    Imagine you are a simple assistant tasked with managing a file system. You have access to three tools:
    
    1. create_dir: Creates a new directory.
    2. create_file: Creates a new file and optionally writes content to it.
    3. read_file: Reads the contents of an existing file.

    Based on the user input and your observations, choose an action to execute. Your action must follow these guidelines:
    
    * Action Guidelines *
    1) Only one action is allowed per iteration.
    2) Be concise and specific about what to create, write, or read.
    3) Provide clear reasoning for your action.
    4) Always consider the current context before taking action.
    
    Respond in the following format:
    Thought: {{Your reasoning here}}
    Action: {{Action name with parameters}}
    
    Available Actions:
    - create_dir [directory_name]
    - create_file [file_name]; [content (optional)]
    - read_file [file_name]
    """

    # Prepare the input for the LLM
    prompt = f"""
    {template}

    User Query: {query}
    Observations: {observations}
    Actions Taken So Far: {actions_taken}
    """

    output = llm_agent.invoke(prompt)

    if output.tool_calls:
        for tool_call in output.tool_calls:
            function_name = tool_call["name"]
            function_args = tool_call.get("args", [])  # default to empty list if "args" is missing

            # Check if function_name exists in tool_mapping
            if function_name not in tool_mapping:
                print(f"Error: Unknown function name '{function_name}'")
                continue

            try:
                # Attempt to call the function with the provided arguments
                tool_output = tool_mapping[function_name](**function_args)
                actions_taken.append(function_name)
                observations.append(tool_output)
            except TypeError as e:
                # Handle errors if the number of arguments does not match the expected number for the function
                print(f"Error: Invalid arguments for function '{function_name}': {e}")
    
    
    return output, actions_taken, observations

In [69]:
llm_call("Create a folder named: lucas-tests-tool-calling")

(AIMessage(content='', response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T22:21:11.784171Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_dir', 'arguments': {'folder_path': 'lucas-tests-tool-calling'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1901656625, 'load_duration': 568133000, 'prompt_eval_count': 558, 'prompt_eval_duration': 451000000, 'eval_count': 55, 'eval_duration': 553000000}, id='run-204906b5-3c97-4285-8b54-4c376a042e9e-0', tool_calls=[{'name': 'create_dir', 'args': {'folder_path': 'lucas-tests-tool-calling'}, 'id': 'bf8293d0-58b8-4be5-b3b6-24738374640e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 558, 'output_tokens': 55, 'total_tokens': 613}),
 ['create_dir'],
 ['Folder path was created at: lucas-tests-tool-calling'])

In [59]:
llm_call("Create a file at: ./lucas-tests-tool-calling/test1.txt with the contents: Hello Lucas! You just made a tool call!")

{'output': ('observation',
  ['A file was created at: ./lucas-tests-tool-calling/test1.txt'])}

In [60]:
llm_call("Read the file contents from  ./lucas-tests-tool-calling/test1.txt")

{'output': ('observation', ['Hello Lucas! You just made a tool call!'])}

In [70]:
def agent_loop(query):
    iter_count = 0
    obs = []
    acts_taken = []
    while True:
        output,obs,acts_taken = llm_call(query, obs, acts_taken)
        print(output)
        iter_count+=1
        if iter_count>=3:
            print(f"Breaking after {iter_count} iterations")
            break
        if output.content!="":
            break
    
    return output.content


agent_loop("Create a folder in current directory named 'testing-multiple-calls' and inside that folder create a file named nested-file.txt")        

content='' response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T22:21:26.57492Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_dir', 'arguments': {'folder_path': './testing-multiple-calls'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 566164709, 'load_duration': 32140750, 'prompt_eval_count': 572, 'prompt_eval_duration': 136000000, 'eval_count': 30, 'eval_duration': 396000000} id='run-b03dcbe3-504f-4a74-896a-9f11ba4fadb4-0' tool_calls=[{'name': 'create_dir', 'args': {'folder_path': './testing-multiple-calls'}, 'id': '8f81c62c-0e25-4a4c-a377-e6732748866c', 'type': 'tool_call'}] usage_metadata={'input_tokens': 572, 'output_tokens': 30, 'total_tokens': 602}
content='' response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T22:21:27.36701Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_dir', 'arguments': {'folder_path': './testing-multiple-calls'}}}]}, 'done

''

Remaining:
- [ ] Integrate a system message so the LLM knows when to stop solving the problem
- [ ] Fix the prompt template stuff from the beggining
- [ ] Fix the redundant looping issues